# Crawlin Image from google

### [cell1] 검색엔진에서 입력한 키워드의 이미지를 찾아 자동으로 임시 폴더(temp)에 다운로드
### [cell2] temp의 이미지파일을 7:3 비율로 Train Set, Test Set으로 분할하여 검색어 이름의 폴더로 분할한다.

#### v1.1 : PATH 안정화
#### v1.2 : tqdm을 이용한 진행도 표시
#### v2.0 : 키워드 리스트 저장후 멀티 크롤링 가능

세원아토스 기술연구소 조영탁

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

import json
import os
import argparse

import requests
import urllib
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import datetime
import time

urllib3.disable_warnings(InsecureRequestWarning)

searchword = 'cat'
searchurl = 'https://www.google.com/search?q=' + searchword + '&source=lnms&tbm=isch'
pdirs = 'dataset'
temp_dirs = 'temp'
maxcount = 1000
os.chdir('C:\Projects\keras_talk')

chromedriver = 'C://Program Files//chromedriver//chromedriver.exe'

if not os.path.exists(temp_dirs):
    os.mkdir(temp_dirs)
    
if not os.path.exists(pdirs):
    os.mkdir(pdirs)

def download_google_staticimages():

    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    #options.add_argument('--headless')

    try:
        browser = webdriver.Chrome(chromedriver, options=options)
    except Exception as e:
        print(f'No found chromedriver in this environment.')
        print(f'Install on your machine. exception: {e}')
        sys.exit()

    browser.set_window_size(1280, 1024)
    browser.get(searchurl)
    time.sleep(1)

    print(f'Scrolling...')

    element = browser.find_element_by_tag_name('body')

    # Scroll down
    #for i in range(30):
    for i in range(50):
        element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)

    try:
        browser.find_element_by_id('smb').click()
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
    except:
        for i in range(10):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

    print(f'페이지 끝에 도달했습니다.')
    time.sleep(0.5)
    print(f'결과 더보기로 재시작합니다.')
    time.sleep(0.5)

    browser.find_element_by_xpath('//input[@value="결과 더보기"]').click()

    # Scroll down 2
    for i in range(50):
        element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)

    try:
        browser.find_element_by_id('smb').click()
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
    except:
        for i in range(10):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

    page_source = browser.page_source 

    soup = BeautifulSoup(page_source, 'lxml')
    images = soup.find_all('img')

    urls = []
    for image in images:
        try:
            url = image['data-src']
            if not url.find('https://'):
                urls.append(url)
        except:
            try:
                url = image['src']
                if not url.find('https://'):
                    urls.append(image['src'])
            except Exception as e:
                print(f'No found image sources.')
                print(e)

    count = 0
    if urls:
        for url in tqdm(urls):
            try:
                res = requests.get(url, verify=False, stream=True)
                rawdata = res.raw.read()
                with open(os.path.join(temp_dirs, 'img_' + str(count) + '.jpg'), 'wb') as f:
                    f.write(rawdata)
                    count += 1
            except Exception as e:
                print('Failed to write rawdata.')
                print(e)

    browser.close()
    return count

# Main block
def main():
    t0 = time.time()
    count = download_google_staticimages()
    t1 = time.time()

    total_time = t1 - t0
    os.chdir('C:\Projects\keras_talk')
    print(f'\n')
    print(f'크롤링 완료. [이미지 수 = {count}].')
    print(f'총 소요 시간 : {str(total_time)} seconds.')

if __name__ == '__main__':
    main()

Scrolling...
페이지 끝에 도달했습니다.
결과 더보기로 재시작합니다...





크롤링 완료. [이미지 수 = 813].
총 소요 시간 : 165.95001435279846 seconds.


In [11]:
import glob


if not os.path.exists(pdirs + '/train/' +  searchword):
    os.mkdir(pdirs + '/train/' +  searchword)
    
if not os.path.exists(pdirs + '/test/' +  searchword):
    os.mkdir(pdirs + '/test/' +  searchword)
    
os.chdir('C:/Projects/keras_talk/' + temp_dirs)   
img_list = glob.glob('*.*')
os.chdir('C:/Projects/keras_talk/')  

print('총 이미지 파일 : ' + str(len(img_list)))
print('train : 1 ~ ' + str(round(len(img_list) * 0.7))) # 1 = 구글 로고 제외

for img in tqdm(img_list[1:round(len(img_list) * 0.7)+1]):
    os.rename(temp_dirs + '/' + img, pdirs + '/train/' +  searchword + '/' + img)

print('test : ' + str(round(len(img_list) * 0.7) + 1) + '~' + str(len(img_list)))
for img in tqdm(img_list[round(len(img_list) * 0.7) + 1 : len(img_list)]):
    os.rename(temp_dirs + '/' + img, pdirs + '/test/' + searchword + '/' + img)

print('TrainSet, TestSet 자동 분류 -> OK')
os.chdir('C:/Projects/keras_talk/' + temp_dirs)   
os.remove(img_list[0])
os.chdir('../')
os.rmdir(temp_dirs)
print('Temp Directory, Logo Data 제거 -> OK')


총 이미지 파일 : 813
train : 1 ~ 569


img_1.jpg
img_10.jpg
img_100.jpg
img_101.jpg
img_102.jpg
img_103.jpg
img_104.jpg
img_105.jpg
img_106.jpg
img_107.jpg
img_108.jpg
img_109.jpg
img_11.jpg
img_110.jpg
img_111.jpg
img_112.jpg
img_113.jpg
img_114.jpg
img_115.jpg
img_116.jpg
img_117.jpg
img_118.jpg
img_119.jpg
img_12.jpg
img_120.jpg
img_121.jpg
img_122.jpg
img_123.jpg
img_124.jpg
img_125.jpg
img_126.jpg
img_127.jpg
img_128.jpg
img_129.jpg
img_13.jpg
img_130.jpg
img_131.jpg
img_132.jpg
img_133.jpg
img_134.jpg
img_135.jpg
img_136.jpg
img_137.jpg
img_138.jpg
img_139.jpg
img_14.jpg
img_140.jpg
img_141.jpg
img_142.jpg
img_143.jpg
img_144.jpg
img_145.jpg
img_146.jpg
img_147.jpg
img_148.jpg
img_149.jpg
img_15.jpg
img_150.jpg
img_151.jpg
img_152.jpg
img_153.jpg
img_154.jpg
img_155.jpg
img_156.jpg
img_157.jpg
img_158.jpg
img_159.jpg
img_16.jpg
img_160.jpg
img_161.jpg
img_162.jpg
img_163.jpg
img_164.jpg
img_165.jpg
img_166.jpg
img_167.jpg
img_168.jpg
img_169.jpg
img_17.jpg
img_170.jpg
img_171.jpg
img_172.jpg
img_173.jpg
img_174.jpg
im

img_611.jpg
img_612.jpg
img_613.jpg
img_614.jpg
img_615.jpg
img_616.jpg
img_617.jpg
img_618.jpg
img_619.jpg
img_62.jpg
img_620.jpg
img_621.jpg
img_622.jpg
img_623.jpg
img_624.jpg
img_625.jpg
img_626.jpg
img_627.jpg
img_628.jpg
img_629.jpg
img_63.jpg
img_630.jpg
img_631.jpg
img_632.jpg
img_633.jpg
img_634.jpg
img_635.jpg
img_636.jpg
img_637.jpg
img_638.jpg
img_639.jpg
img_64.jpg
img_640.jpg
img_641.jpg
img_642.jpg
img_643.jpg
img_644.jpg
img_645.jpg
img_646.jpg
img_647.jpg
img_648.jpg
img_649.jpg
img_65.jpg
img_650.jpg
img_651.jpg
img_652.jpg
img_653.jpg
img_654.jpg
img_655.jpg
img_656.jpg
img_657.jpg
img_658.jpg
img_659.jpg
img_66.jpg
img_660.jpg
img_661.jpg
img_662.jpg
img_663.jpg
img_664.jpg
img_665.jpg
img_666.jpg
img_667.jpg
img_668.jpg
img_669.jpg
img_67.jpg
img_670.jpg
img_671.jpg
img_672.jpg
img_673.jpg
img_674.jpg
img_675.jpg
img_676.jpg
img_677.jpg
img_678.jpg
img_679.jpg
img_68.jpg
img_680.jpg
img_681.jpg
img_682.jpg
img_683.jpg
img_684.jpg
img_685.jpg
img_686.jpg
img_687.jpg